In [1]:
!pip install --upgrade pip
!pip install optuna

In [2]:
import pandas as pd
import random as rd
import xgboost as xgb
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from tqdm.notebook import tqdm_notebook
from pathlib import Path 

pd.set_option('display.max_columns', None)

### 1. Load the Data

In [6]:
# def label_3_category(x) :
#   if x==65 or x==66 :
#     return 0
#   elif x==67 or x==68:
#     return 1
#   elif x==69 or x==70 :
#     return 2

# def label_emotions(x) :
#   if x==65 or x==67 or x==69:
#     return 0
#   elif x==66 or x==68 or x==70:
#     return 1
def label_2by2(x) :
  if x==65:
    return 0
  elif x==66:
    return 1
# def label_subj(x) :
#   if x<=0:
#     return 0
#   elif x>=0.01:
#     return 1

In [9]:
df = pd.read_csv("pathtodataset.csv", sep=';')
# df = df.drop(df[(df.Condition == 67) | (df.Condition == 68)].index, axis=0)
print(df)
df.drop(['pose1', '_30'], axis=1, inplace=True)
# df.drop(['Emotion', 'Presence'], axis=1, inplace=True)
df.drop(['Emotion'], axis=1, inplace=True)
# df['label_3_category'] = df['Condition'].apply(lambda x : label_3_category(x))
# df['label_emotions'] = df['Condition'].apply(lambda x : label_emotions(x))
df['label_2by2'] = df['Condition'].apply(lambda x : label_2by2(x))
# df['label_subj'] = df['Presence'].apply(lambda x : label_subj(x))

       Subject Condition   Emotion         pose0  pose1     pose2     pose3  \
0       CHE324  SelfStim  0.717979  9.620000e-06      0  0.000000  0.000092   
1       CHE324  SelfStim  0.717979  4.814770e-03      0  0.004836  0.003188   
2       CHE324  SelfStim  0.717979  6.820000e-07      0  0.000000  0.000050   
3       CHE324  SelfStim  0.717979  1.473792e-03      0  0.001650  0.003636   
4       CHE324  SelfStim  0.717979  4.880052e-03      0  0.000000  0.000020   
...        ...       ...       ...           ...    ...       ...       ...   
662927  TAI310  CtrlStim  0.106771  7.610000e-06      0  0.000000  0.000022   
662928  TAI310  CtrlStim  0.106771  2.660000e-05      0  0.000000  0.000072   
662929  TAI310  CtrlStim  0.106771  3.040000e-05      0  0.000000  0.000029   
662930  TAI310  CtrlStim  0.106771  2.280000e-05      0  0.000000  0.000108   
662931  TAI310  CtrlStim  0.106771  2.280000e-05      0  0.000000  0.000007   

           pose4     pose5     pose6     pose7    p

In [10]:
def normalize_df (df):
  # For first patient
  df_normalized = df[df['Subject']=='CHE324']
  scaler = MinMaxScaler()
  df_normalized.iloc[:,2:-1] = scaler.fit_transform(df_normalized.iloc[:,2:-1])

  # For the rest
  subject_ids = list(df.Subject.unique())[1:]
  for subject in subject_ids :
    df_subject = df[df['Subject']==subject]
    scaler = MinMaxScaler()
    df_subject.iloc[:,2:-1] = scaler.fit_transform(df_subject.iloc[:,2:-1])
    df_normalized = df_normalized.append(df_subject)
  
  return df_normalized

In [11]:
df_normalized = normalize_df(df)

C:\Users\Bruno\AppData\Local\Temp\ipykernel_20260\4125737943.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normalized.iloc[:,2:-1] = scaler.fit_transform(df_normalized.iloc[:,2:-1])
C:\Users\Bruno\AppData\Local\Temp\ipykernel_20260\4125737943.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subject.iloc[:,2:-1] = scaler.fit_transform(df_subject.iloc[:,2:-1])
C:\Users\Bruno\AppData\Local\Temp\ipykernel_20260\4125737943.py:13: FutureWarning: The frame.append method is deprecated and will be re

C:\Users\Bruno\AppData\Local\Temp\ipykernel_20260\4125737943.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_normalized = df_normalized.append(df_subject)
C:\Users\Bruno\AppData\Local\Temp\ipykernel_20260\4125737943.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subject.iloc[:,2:-1] = scaler.fit_transform(df_subject.iloc[:,2:-1])
C:\Users\Bruno\AppData\Local\Temp\ipykernel_20260\4125737943.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_normalized = df_normalized.append(df_subject)
C:\Users\Bruno\AppData\Local\Temp\ipykernel_20260\4125737943.py:12

### 2. Preprocessing

In [12]:
def split_df (valid_ids, test_ids, df_normalized = df_normalized):
  print('splitting dataframe ...')
  df_train = df_normalized[~df_normalized['Subject'].isin(valid_ids+test_ids)]
  df_valid = df_normalized[df_normalized['Subject'].isin(valid_ids)]
  df_test = df_normalized[df_normalized['Subject'].isin(test_ids)]
  return df_train, df_valid, df_test

### 3. Creation of the MA features and Split

In [13]:
def create_dataset (df_train, df_valid, df_test, win_size = 1000, labels = 'label_2by2'):
  print('creating MA dataset ...')
  # For train set:
  df_train_MA = df_train.groupby(by=['Condition', 'Subject']).rolling(win_size).mean() # moving average over 30 points

  df_train_MA.dropna(inplace=True) # drop the na rows created
  df_train_MA = df_train_MA.reset_index() # make the Condition as column
  df_train_MA.drop(columns=df_train_MA.columns[2], axis=1, inplace=True) # remove the column created with reset_index()

  X_train = df_train_MA.iloc[:, 2:-1]
  y_train = df_train_MA[labels]

  # For validation set:
  df_valid_MA = df_valid.groupby(by=['Condition', 'Subject']).rolling(win_size).mean() # moving average over 30 points

  df_valid_MA.dropna(inplace=True) # drop the na rows created
  df_valid_MA = df_valid_MA.reset_index() # make the Condition as column
  df_valid_MA.drop(columns=df_valid_MA.columns[2], axis=1, inplace=True) # remove the column created with reset_index()

  X_valid = df_valid_MA.iloc[:, 2:-1]
  y_valid = df_valid_MA[labels]

  # For test set :
  df_test_MA = df_test.groupby(by=['Condition', 'Subject']).rolling(win_size).mean() # moving average over 30 points

  df_test_MA.dropna(inplace=True) # drop the na rows created
  df_test_MA = df_test_MA.reset_index() # make the Condition as column
  df_test_MA.drop(columns=df_test_MA.columns[2], axis=1, inplace=True) # remove the column created with reset_index()

  X_test = df_test_MA.iloc[:, 2:-1]
  y_test = df_test_MA[labels]

  return X_train, y_train, X_valid, y_valid, X_test, y_test


### 4. Definiton and tunning the model

In [14]:
def objective(trial, X_train, y_train, X_valid, y_valid):
    
    param = {
        'max_depth': trial.suggest_int('max_depth', 5, 14),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 15000, 100),
        'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
        'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        }
   
    model = XGBClassifier(random_state=42,
                          objective='multi:softmax',
                          num_class = 2,
                          eval_metric='mlogloss',
                          use_label_encoder=False,
                          tree_method='gpu_hist',
                          **param ) 
    
    model.fit(X_train, y_train,eval_set=[(X_valid, y_valid)], early_stopping_rounds=150,verbose=False)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    return acc

### 5. Feature importance

In [15]:
def save_features (model, df, filename):
  features = list(df.columns.values[2:-1])
  feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_, features)), columns=['Value','Feature'])
  #save to csv
  #feature_imp.to_csv("outputpath" + filename + "nameofcontrast.csv")

#### Cross Validation

In [16]:
def get_valid_train_ids (subject, subject_ids):

  print('get ids ...')
  valid_ids = []
  test_ids = []
  valid_ids.append(subject_ids[subject])
#   valid_ids.append(subject_ids[(subject + 1)%16])

  test_ids.append(subject_ids[(subject + 1)%16])
#   test_ids.append(subject_ids[(subject + 3)%15])

  return valid_ids, test_ids

#### Optuna

In [17]:
def get_best_params (X_train, y_train, X_valid, y_valid, n_trials = 10):
  study = optuna.create_study(direction='maximize')

  objective_with_param = lambda trial: objective(trial, X_train, y_train, X_valid, y_valid)

  study.optimize(objective_with_param, n_trials=n_trials)

  return study.best_params

### 6. Train & Test

In [18]:
def cross_valid ():
    results = []
    subject_ids = list(df.Subject.unique())

    for sub in range(0, 16, 1):
        valid_ids, test_ids = get_valid_train_ids(sub, subject_ids)

        df_train, df_valid, df_test = split_df(valid_ids, test_ids)

        X_train, y_train, X_valid, y_valid, X_test, y_test = create_dataset(df_train, df_valid, df_test, 1000, labels='label_2by2')

        best_params = get_best_params (X_train, y_train, X_valid, y_valid)

        model_optimal = XGBClassifier(random_state=42,
                                objective='multi:softmax',
                                num_class = 2,
                                eval_metric='mlogloss',
                                use_label_encoder=False,
                                tree_method='gpu_hist',
                                **best_params) 
        print('training ...')
        model_optimal.fit(X_train, y_train)


        print('predicting ...')
        y_pred = model_optimal.predict(X_test)
        acc_test = accuracy_score(y_pred, y_test)

        valid_ids_str = '+'.join(map(str, valid_ids))
        test_ids_str = '+'.join(map(str, test_ids))

        print("The accuracy on " + test_ids_str + " is :{:2f} .".format(acc_test))
        #save the results with the list of subject (in the title)
        results.append((valid_ids_str + '_' + test_ids_str, acc_test))

        #save the feature importances
        print('saving ...')
        save_features(model_optimal, df_train, valid_ids_str + test_ids_str)
    
    print('cross validation finished')
    return results


### 7. Results

In [19]:
results = cross_valid()

get ids ...
splitting dataframe ...
creating MA dataset ...


[I 2023-07-11 10:32:22,206] A new study created in memory with name: no-name-984faba1-c90d-4ee3-a2ea-c9bbd223adee
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-07-11 10:32:25,367] Trial 0 finished with value: 0.5727583262169086 and parameters: {'max_depth': 8, 'n_estimators': 11200, 'reg_alpha': 11, 'reg_lambda': 77, 'min_child_weight': 16, 'colsample_bytree': 0.2185764910387345}. Best is trial 0 with value: 0.5727583262169086.
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages

training ...
predicting ...
The accuracy on DES316 is :0.577306 .
saving ...
get ids ...
splitting dataframe ...
creating MA dataset ...


[I 2023-07-11 10:33:52,524] A new study created in memory with name: no-name-e917a21b-8f47-4bb9-9fb8-88ef08b06ac2
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-07-11 10:33:54,876] Trial 0 finished with value: 0.6449900464499004 and parameters: {'max_depth': 8, 'n_estimators': 5300, 'reg_alpha': 7, 'reg_lambda': 19, 'min_child_weight': 11, 'colsample_bytree': 0.6605660238713746}. Best is trial 0 with value: 0.6449900464499004.
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\x

training ...
predicting ...
The accuracy on EDC312 is :0.557842 .
saving ...
get ids ...
splitting dataframe ...
creating MA dataset ...


...

training ...
predicting ...
The accuracy on SAE308 is :0.384265 .
saving ...
get ids ...
splitting dataframe ...
creating MA dataset ...


[I 2023-07-11 10:48:48,069] A new study created in memory with name: no-name-c1c7e8db-e0db-4acb-8837-c253c596f233
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-07-11 10:48:50,564] Trial 0 finished with value: 0.3960794609258773 and parameters: {'max_depth': 10, 'n_estimators': 3500, 'reg_alpha': 26, 'reg_lambda': 22, 'min_child_weight': 4, 'colsample_bytree': 0.15308929255547044}. Best is trial 0 with value: 0.3960794609258773.
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages

training ...
predicting ...
The accuracy on TAI310 is :0.723770 .
saving ...
get ids ...
splitting dataframe ...
creating MA dataset ...


[I 2023-07-11 10:49:41,155] A new study created in memory with name: no-name-9abe03a4-b88d-4cad-9f8f-ada630e855c9
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-07-11 10:49:43,325] Trial 0 finished with value: 0.7136876281613124 and parameters: {'max_depth': 5, 'n_estimators': 11100, 'reg_alpha': 43, 'reg_lambda': 50, 'min_child_weight': 13, 'colsample_bytree': 0.4199572841910665}. Best is trial 0 with value: 0.7136876281613124.
C:\Users\Bruno\Documents\PycharmProjects\scripts_exp\psat\venv\lib\site-packages

training ...
predicting ...
The accuracy on CHE324 is :0.449530 .
saving ...
cross validation finished


In [28]:
results = np.array(results)
print(results)
mean = np.mean(results[:,1], dtype= np.float32)
df = pd.DataFrame(results, columns = ['Fold','Accuracy'])
#save results
df.to_csv('outputpath.csv', sep=',', index=False)
print(mean)

[0.57730591 0.55784195 0.7412077  0.91867268 0.61346489 0.54062893
 0.77417348 0.56146122 0.69523087 0.80874631 0.50205592 0.98388309
 0.55954027 0.38426534 0.72376965 0.44953032]


ValueError: Shape of passed values is (16, 1), indices imply (16, 2)

## ----END---- ##